In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import psycopg2
from psycopg2.extras import RealDictCursor


In [ ]:
def connect_to_postgres(dbname='postgres_db'):
    """Connect to the PostgreSQL database."""
    conn = psycopg2.connect(
        dbname=dbname,
        user='postgres',
        password='password',
        host='localhost',
        port=5432,
    )
    conn.autocommit = False  # Ensure transactions are used
    return conn


def sql_fetch(sql):
    conn = connect_to_postgres()
    conn.autocommit = True
    cursor = conn.cursor(cursor_factory=RealDictCursor)

    try:
        cursor.execute(sql)
        results = cursor.fetchall()
    finally:
        cursor.close()
        conn.close()

    return results

In [ ]:
sql = """
WITH migros_ids_with_5_occurrences AS (
    SELECT
        p.migros_id
    FROM product p
    GROUP BY p.migros_id
    HAVING COUNT(*) = 5
)
SELECT
    p.migros_id,
    o.unit_price,
    p.scraped_at
FROM product p
JOIN nutrients n ON n.id = p.nutrient_id
JOIN offer o ON p.offer_id = o.id
WHERE p.migros_id IN (SELECT migros_id FROM migros_ids_with_5_occurrences)
ORDER BY p.migros_id, p.scraped_at;
"""

sql = """
SELECT
    p.migros_id,
    p.name,
    o.unit_price,
    p.scraped_at
FROM product p
JOIN offer o ON p.offer_id = o.id
WHERE p.migros_id = '220220030020'
ORDER BY p.scraped_at;
"""
res = sql_fetch(sql)
pprint(res)

df = pd.DataFrame(res)

pprint(df.columns)


# Check if the DataFrame is empty
if df.empty:
    print("No data found for products with exactly 5 occurrences.")
else:
    # Convert scraped_at to datetime
    df['scraped_at'] = pd.to_datetime(df['scraped_at'])

    # Plot each migros_id's unit_price over time
    for migros_id in df['migros_id'].unique():
        product_data = df[df['migros_id'] == migros_id]
        plt.plot(product_data['scraped_at'], product_data['unit_price'], marker='o', label=f'Migros ID {migros_id}')

    # Plot settings
    plt.xlabel('Scraped At')
    plt.ylabel('Unit Price')
    plt.title('Price Changes for Products with Exactly 5 Occurrences')
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()

    # Show plot
    plt.show()


tablespoon

In [ ]:
sql = """
WITH product_price_data AS (
    SELECT
        p.migros_id,
        p.name,
        o.unit_price,
        p.scraped_at,
        ROW_NUMBER() OVER (PARTITION BY p.migros_id ORDER BY p.scraped_at ASC) AS row_num_asc,
        ROW_NUMBER() OVER (PARTITION BY p.migros_id ORDER BY p.scraped_at DESC) AS row_num_desc
    FROM product p
    JOIN offer o ON p.offer_id = o.id
)
, price_changes AS (
    SELECT
        initial.migros_id,
        initial.name,
        initial.unit_price AS initial_price,
        latest.unit_price AS current_price,
        (latest.unit_price - initial.unit_price) / initial.unit_price * 100 AS percentage_difference
    FROM product_price_data initial
    JOIN product_price_data latest
      ON initial.migros_id = latest.migros_id
    WHERE initial.row_num_asc = 1
      AND latest.row_num_desc = 1
)
SELECT
    migros_id,
    name,
    initial_price,
    current_price,
    percentage_difference
FROM price_changes
ORDER BY percentage_difference DESC;
"""

res = sql_fetch(sql)
df = pd.DataFrame(res)


df_filtered = df.dropna(subset=['initial_price', 'current_price'])

# Ensure that only the top 5 products with valid data are selected
df_top_5 = df_filtered.sort_values(by='percentage_difference', ascending=False).head(5)
pprint(df_top_5)


df_top_5.loc[8173, 'initial_price'] = df_top_5.loc[8173, 'initial_price']/60
df_top_5.loc[8173, 'current_price'] = df_top_5.loc[8173, 'current_price']/60


df_top_5.loc[8176, 'initial_price'] = 54.95 /24
df_top_5.loc[8176, 'current_price'] = 80.85/24


df_top_5.loc[8177, 'initial_price'] = 44.95/24
df_top_5.loc[8177, 'current_price'] = 65.85/24

pprint(df_top_5)

# Check if filtered data is empty
if df_top_5.empty:
    print("No data found with both initial and current prices available.")
else:
    # Plot the percentage price changes for the top 5 products
    fig, ax = plt.subplots(figsize=(10, 6))

    for i, row in df_top_5.iterrows():
        plt.plot(
            ['Initial Price', 'Current Price'],
            [row['initial_price'], row['current_price']],
            marker='o',
            label=f"{row['name']} : {row['percentage_difference']:.0f}%"
        )

    # plt.title('Top 5 Products with Largest Percentage Price Increase (Excluding Incomplete Data)')
    # plt.xlabel('Time Period')
    # plt.ylabel('Price (in currency)')
    plt.legend()
    plt.grid(True)
    plt.show()

import matplotlib.pyplot as plt
import numpy as np

if df_top_5.empty:
    print("No data found with both initial and current prices available.")
else:
    # Create a figure and axis for plotting
    fig, ax = plt.subplots(figsize=(12, 8))

    # Define the bar width
    bar_width = 0.35

    # Get the indices for the products
    indices = np.arange(len(df_top_5))

    # Extract initial and current prices
    initial_prices = df_top_5['initial_price'].astype(float)
    current_prices = df_top_5['current_price'].astype(float)

    # Plot initial prices as bars
    ax.bar(indices - bar_width/2, initial_prices, bar_width, label='Initial Price', color='blue')

    # Plot current prices as bars
    ax.bar(indices + bar_width/2, current_prices, bar_width, label='Current Price', color='orange')

    # Set the x-axis labels to product names
    ax.set_xticks(indices)
    ax.set_xticklabels(df_top_5['name'], rotation=45, ha='right')

    # Set the chart title and labels
    # ax.set_title('Top 5 Products with Largest Percentage Price Change')
    # ax.set_xlabel('Product Name')
    # ax.set_ylabel('Price (in currency)')
    ax.legend()

    # Add a grid for better readability
    ax.grid(axis='y', linestyle='--', linewidth=0.7)

    # Show the plot
    plt.tight_layout()
    plt.show()


In [ ]:
sql = """
WITH product_price_data AS (
    SELECT
        p.migros_id,
        p.name,
        o.unit_price,
        p.scraped_at,
        ROW_NUMBER() OVER (PARTITION BY p.migros_id ORDER BY p.scraped_at ASC) AS row_num_asc,
        ROW_NUMBER() OVER (PARTITION BY p.migros_id ORDER BY p.scraped_at DESC) AS row_num_desc
    FROM product p
    JOIN offer o ON p.offer_id = o.id
)
, price_changes AS (
    SELECT
        initial.migros_id,
        initial.name,
        initial.unit_price AS initial_price,
        latest.unit_price AS current_price,
        ((latest.unit_price - initial.unit_price) / initial.unit_price) * 100 AS percentage_difference
    FROM product_price_data initial
    JOIN product_price_data latest
      ON initial.migros_id = latest.migros_id
    WHERE initial.row_num_asc = 1
      AND latest.row_num_desc = 1
)
SELECT
    migros_id,
    name,
    initial_price,
    current_price,
    percentage_difference
FROM price_changes
WHERE percentage_difference < 0
ORDER BY percentage_difference ASC;
"""

# Fetch the data from the PostgreSQL database
def sql_fetch(sql):
    conn = connect_to_postgres()
    conn.autocommit = True
    cursor = conn.cursor()

    try:
        cursor.execute(sql)
        column_names = [desc[0] for desc in cursor.description]
        results = cursor.fetchall()
        df = pd.DataFrame(results, columns=column_names)
    except Exception as e:
        print(f"Error fetching data: {e}")
        df = pd.DataFrame()
    finally:
        cursor.close()
        conn.close()

    return df

# Load data
df = sql_fetch(sql)

# Filter out rows where either initial_price or current_price is None
df_filtered = df.dropna(subset=['initial_price', 'current_price'])

# Ensure that only the top 5 products with valid data are selected
df_top_5 = df_filtered.sort_values(by='percentage_difference', ascending=True).head(5)
pprint(df_top_5)


df_top_5.loc[0, 'initial_price'] = 3.9
df_top_5.loc[0, 'current_price'] = 1.9



df_top_5.loc[1, 'initial_price'] = round(25.85 / 16, 2)
df_top_5.loc[1, 'current_price'] = round(15.5 / 16, 2)
# round(25.85 / 16, 2)

df_top_5.loc[3, 'name'] = 'chicken breast'


df_top_5.loc[4, 'initial_price'] = 1.75
df_top_5.loc[4, 'current_price'] = 1.25

pprint(df_top_5)

# Check if filtered data is empty
if df_top_5.empty:
    print("No data found with both initial and current prices available.")
else:
    # Plot the percentage price decreases for the top 5 products
    fig, ax = plt.subplots(figsize=(10, 6))

    for i, row in df_top_5.iterrows():
        plt.plot(
            ['Initial Price', 'Current Price'],
            [row['initial_price'], row['current_price']],
            marker='o',
            label=f"{row['name']}: {row['percentage_difference']:.2f}%"
        )

    # plt.title('Top 5 Products with Largest Percentage Price Decrease (Excluding Incomplete Data)')
    # plt.xlabel('Time Period')
    # plt.ylabel('Price (in currency)')
    plt.legend()
    plt.grid(True)
    plt.show()

if df_top_5.empty:
    print("No data found with both initial and current prices available.")
else:
    # Create a figure and axis for plotting
    fig, ax = plt.subplots(figsize=(12, 8))

    # Define the bar width
    bar_width = 0.35

    # Get the indices for the products
    indices = np.arange(len(df_top_5))

    # Extract initial and current prices
    initial_prices = df_top_5['initial_price'].astype(float)
    current_prices = df_top_5['current_price'].astype(float)

    # Plot initial prices as bars
    ax.bar(indices - bar_width/2, initial_prices, bar_width, label='Initial Price', color='blue')

    # Plot current prices as bars
    ax.bar(indices + bar_width/2, current_prices, bar_width, label='Current Price', color='orange')

    # Set the x-axis labels to product names
    ax.set_xticks(indices)
    ax.set_xticklabels(df_top_5['name'], rotation=45, ha='right')

    # Set the chart title and labels
    # ax.set_title('Top 5 Products with Largest Percentage Price Change')
    # ax.set_xlabel('Product Name')
    # ax.set_ylabel('Price (in currency)')
    ax.legend()

    # Add a grid for better readability
    ax.grid(axis='y', linestyle='--', linewidth=0.7)

    # Show the plot
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd

# Fetch data using the SQL query
sql = """
WITH product_price_data AS (
    SELECT
        p.migros_id,
        p.name,
        o.price,
        p.scraped_at,
        ROW_NUMBER() OVER (PARTITION BY p.migros_id ORDER BY p.scraped_at ASC) AS row_num_asc,
        ROW_NUMBER() OVER (PARTITION BY p.migros_id ORDER BY p.scraped_at DESC) AS row_num_desc
    FROM product p
    JOIN offer o ON p.offer_id = o.id
    WHERE p.nutrient_id IS NOT NULL
)
, price_changes AS (
    SELECT
        initial.migros_id,
        initial.name,
        initial.price AS initial_price,
        latest.price AS current_price,
        ((latest.price - initial.price) / initial.price) * 100 AS percentage_difference
    FROM product_price_data initial
    JOIN product_price_data latest
      ON initial.migros_id = latest.migros_id
    WHERE initial.row_num_asc = 1
      AND latest.row_num_desc = 1
      AND initial.price IS NOT NULL
      AND latest.price IS NOT NULL
)
SELECT
    migros_id,
    name,
    initial_price,
    current_price,
    percentage_difference
FROM price_changes;
"""

# Assuming sql_fetch is a function that retrieves the data from the database
df = sql_fetch(sql)

# Convert 'percentage_difference' column to float to avoid scientific notation
df['percentage_difference'] = df['percentage_difference'].astype(float)

# Alternatively, use round to ensure proper formatting if you want to limit precision
df['percentage_difference'] = df['percentage_difference'].round(2)

# Print to verify the output
print(df.head())

# Drop rows with missing values for initial_price or current_price, just in case
df_filtered = df.dropna(subset=['initial_price', 'current_price', 'percentage_difference'])


pprint(df_filtered.head)

# Calculate the average percentage increase
if not df_filtered.empty:
    average_percentage_increase = df_filtered['percentage_difference'].mean()
    median_percentage_increase = df_filtered['percentage_difference'].median()
    spercentages_sum = df_filtered['percentage_difference'].sum()
    print(f"Average Percentage Price Increase: {average_percentage_increase:.2f}%")
    print(f"Median Percentage Price Increase: {median_percentage_increase:.2f}%")
    print(f"Sum of all percentages: {spercentages_sum:.2f}%")
else:
    print("No valid data available for calculating the average price increase.")


In [ ]:
print(0.11 / 64 * 365)
print(0.04 / 64 * 365)
print(80*28)
print(3.75/(80*28/100))
print(7.2/4)